In [ ]:
import numpy as np
import scipy as sp
from scipy.stats import mode
import pandas as pd
import matplotlib.pyplot as plt
from time import time

import librosa
import librosa.display

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.ensemble import BaggingClassifier
from sklearn import linear_model
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import SVC

In [ ]:
def load_sound_file(file_path):
    X, sr = librosa.load(file_path, sr=None)
    return X

def extract_feature(file_name): # Late fusion
    X, sample_rate = librosa.load(file_name, sr=None)
    mfcc = librosa.feature.mfcc(y=X, sr=sample_rate, n_fft=4096, hop_length=4096, n_mfcc=n_mfcc).T
    mfcc_delta = librosa.feature.delta(mfcc, width=5, order=1, trim=True)
    return mfcc, mfcc_delta

def parse_audio_files(file_names, file_labels):
    features, features_delta, labels = np.empty((0,n_mfcc-1)), np.empty((0,n_mfcc)), np.empty(0)
    for fn, fl in zip(file_names, file_labels):
        try:
            mfcc, mfcc_delta = extract_feature(fn)
        except Exception as e:
            print ("Error encountered while parsing file: ", fn)
            continue
        features = np.vstack([features, mfcc])
        features_delta = np.vstack([features_delta, mfcc_delta])
        labels = np.append(labels, fl*np.ones(mfcc.shape[0]))
    return np.array(features), np.array(features_delta), np.array(labels, dtype = np.int)

def predict_proba(clf1, clf2, X_val):
    pred_proba = np.empty((0,30))
    for x in X_val:
        x_mfcc, x_mfcc_delta = extract_feature(x)
        y_pred_proba1 = np.sum(clf1.predict_proba(x_mfcc), axis=0).reshape(-1)
        y_pred_proba2 = np.sum(clf2.predict_proba(x_mfcc_delta), axis=0).reshape(-1)
        pred_proba = np.vstack([pred_proba,np.hstack([y_pred_proba1,y_pred_proba2])])
        #print pred_proba.shape
    return np.array(pred_proba, dtype=np.int)




In [ ]:
def plot_wave(title, raw_sound):
    plt.close('all')
    plt.figure(figsize=(20,6))
    plt.title(title)
    librosa.display.waveplot(np.array(raw_sound), sr=16000)
    plt.show()
    
def plot_specgram(title,raw_sound):
    plt.close('all')
    plt.figure(figsize=(20,6))
    plt.title(title)
    plt.specgram(np.array(raw_sound), Fs=16000)
    plt.show()

def plot_log_power_specgram(title,raw_sound):
    plt.close('all')
    plt.figure(figsize=(20,6))
    plt.title(title)
    D = librosa.logamplitude(np.abs(librosa.stft(raw_sound))**2, ref_power=np.max)
    librosa.display.specshow(D, x_axis='time' ,y_axis='log')
    plt.specgram(np.array(f), Fs=16000)
    plt.show()

In [ ]:
from functools import partial
from pathlib import Path
from multiprocessing import Pool
import os
import shutil
import numpy as np
import pandas as pd
import librosa
from scipy.io import wavfile
from tqdm import tqdm_notebook as tqdm
import torch.nn.functional as F
from fastai.basic_data import DatasetType


In [ ]:
DATA = Path('/content/drive/MyDrive/ADReSS-IS2020-train')
NSYNTH_AUDIO = DATA/'ADReSS-IS2020-data'
TRAIN_AUDIO_PATH = NSYNTH_AUDIO/'train/Full_wave_enhanced_audio/cd'
TRAIN_AUDIO_PATH2 = NSYNTH_AUDIO/'train/Full_wave_enhanced_audio/cc'

NSYNTH_IMAGES = DATA/'ADReSS2020_images'
TRAIN_IMAGE_PATH = NSYNTH_IMAGES/'train'
train_acoustic_fnames = [f.name for f in (TRAIN_AUDIO_PATH.iterdir())
                         if 'S' in f.name]

len(train_acoustic_fnames)

54

In [ ]:
train_acoustic_fnames.append([f.name for f in (TRAIN_AUDIO_PATH2.iterdir())
                         if 'S' in f.name])


In [ ]:
fcd = [f.name for f in (TRAIN_AUDIO_PATH.iterdir())
                         if 'S' in f.name]

len(fcd)

fcc = [f.name for f in (TRAIN_AUDIO_PATH2.iterdir())
                         if 'S' in f.name]

len(fcc)

54

In [ ]:
len(fcc)

54

In [ ]:
train_fnames=[]
for i in fcc:
   train_fnames.append(i)
for i in fcd:
   train_fnames.append(i)

In [ ]:
train_fnames

['S001.wav',
 'S002.wav',
 'S003.wav',
 'S004.wav',
 'S005.wav',
 'S006.wav',
 'S007.wav',
 'S009.wav',
 'S011.wav',
 'S012.wav',
 'S013.wav',
 'S015.wav',
 'S016.wav',
 'S017.wav',
 'S018.wav',
 'S019.wav',
 'S020.wav',
 'S021.wav',
 'S024.wav',
 'S025.wav',
 'S027.wav',
 'S028.wav',
 'S029.wav',
 'S030.wav',
 'S032.wav',
 'S033.wav',
 'S034.wav',
 'S035.wav',
 'S036.wav',
 'S038.wav',
 'S039.wav',
 'S040.wav',
 'S041.wav',
 'S043.wav',
 'S048.wav',
 'S049.wav',
 'S051.wav',
 'S052.wav',
 'S055.wav',
 'S056.wav',
 'S058.wav',
 'S059.wav',
 'S061.wav',
 'S062.wav',
 'S063.wav',
 'S064.wav',
 'S067.wav',
 'S068.wav',
 'S070.wav',
 'S071.wav',
 'S072.wav',
 'S073.wav',
 'S076.wav',
 'S077.wav',
 'S079.wav',
 'S080.wav',
 'S081.wav',
 'S082.wav',
 'S083.wav',
 'S084.wav',
 'S086.wav',
 'S087.wav',
 'S089.wav',
 'S090.wav',
 'S092.wav',
 'S093.wav',
 'S094.wav',
 'S095.wav',
 'S096.wav',
 'S097.wav',
 'S100.wav',
 'S101.wav',
 'S103.wav',
 'S104.wav',
 'S107.wav',
 'S108.wav',
 'S110.wav',

In [ ]:
def read_file(filename, path='', sample_rate=None, trim=False):
    filename = Path(path) / filename
    file_sr, data = wavfile.read(filename)
    if data.dtype == np.int16:
        data = np.float32(data) / np.iinfo(np.int16).max
    elif data.dtype != np.float32:
        raise OSError('Encounted unexpected dtype: {}'.format(data.dtype))
    if sample_rate is not None and sample_rate != file_sr:
        if len(data) > 0:
            data = librosa.core.resample(data, file_sr, sample_rate, res_type='kaiser_fast')
        file_sr = sample_rate
    if trim and len(data) > 1:
        data = librosa.effects.trim(data, top_db=40)[0]
    return data, file_sr

In [ ]:
def mfcc_extraction(fname, src_path):
    #X, sample_rate = librosa.load(file_name, sr=None)
    X, sample_rate = read_file(fname, src_path)
    #mfcc = librosa.feature.mfcc(y=librosa.effects.harmonic(X), 
    #sr=sample_rate, n_fft=4096, hop_length=2048, n_mfcc=n_mfcc).T
    mfcc = librosa.feature.mfcc(y=X, sr=sample_rate, n_fft=4096, hop_length=4096, n_mfcc=n_mfcc).T
    mfcc_delta = librosa.feature.delta(mfcc, width=5, order=1)
    mfcc_deltadelta = librosa.feature.delta(mfcc,width=5, order=2)
    len(mfcc)
    len(mfcc_delta)
    return mfcc, mfcc_delta,mfcc_deltadelta 
    
    
    '''
    x, sample_rate = read_file(fname, src_path)
    
    n_fft = 1024
    hop_length = 256
    n_mels = 40
    fmin = 20
    fmax = sample_rate / 2 
    
    mel_spec_power = librosa.feature.melspectrogram(x, sr=sample_rate, n_fft=n_fft, 
                                                    hop_length=hop_length, 
                                                    n_mels=n_mels, power=2.0, 
                                                    fmin=fmin, fmax=fmax)
    mel_spec_db = librosa.power_to_db(mel_spec_power, ref=np.max)
    dst_fname = dst_path / (fname[:-4] + '.png')
    plt.imsave(dst_fname, mel_spec_db)
    '''

In [ ]:
import glob
import numpy as np
control = glob.glob('/content/drive/MyDrive/ADReSS-IS2020-train/ADReSS-IS2020-data/train/Full_wave_enhanced_audio/cc/*.*')
dementia = glob.glob('/content/drive/MyDrive/ADReSS-IS2020-train/ADReSS-IS2020-data/train/Full_wave_enhanced_audio/cd/*.*')

import tensorflow as tf
#data = []
labels = []
for i in control:   
    labels.append('Control')
for i in dementia:   
    labels.append('Dementia')

#train_data = np.array(data)
#train_labels = np.array(labels)

In [ ]:
labels

['Control',
 'Control',
 'Control',
 'Control',
 'Control',
 'Control',
 'Control',
 'Control',
 'Control',
 'Control',
 'Control',
 'Control',
 'Control',
 'Control',
 'Control',
 'Control',
 'Control',
 'Control',
 'Control',
 'Control',
 'Control',
 'Control',
 'Control',
 'Control',
 'Control',
 'Control',
 'Control',
 'Control',
 'Control',
 'Control',
 'Control',
 'Control',
 'Control',
 'Control',
 'Control',
 'Control',
 'Control',
 'Control',
 'Control',
 'Control',
 'Control',
 'Control',
 'Control',
 'Control',
 'Control',
 'Control',
 'Control',
 'Control',
 'Control',
 'Control',
 'Control',
 'Control',
 'Control',
 'Control',
 'Dementia',
 'Dementia',
 'Dementia',
 'Dementia',
 'Dementia',
 'Dementia',
 'Dementia',
 'Dementia',
 'Dementia',
 'Dementia',
 'Dementia',
 'Dementia',
 'Dementia',
 'Dementia',
 'Dementia',
 'Dementia',
 'Dementia',
 'Dementia',
 'Dementia',
 'Dementia',
 'Dementia',
 'Dementia',
 'Dementia',
 'Dementia',
 'Dementia',
 'Dementia',
 'Dementia',
 

In [ ]:
labels_train = pd.factorize(labels)[0]

In [ ]:
labels_train

array([0, 0, 0, 0, ..., 1, 1, 1, 1])

In [ ]:
train_labels = np.array(labels)
import keras
from sklearn import preprocessing
lb = preprocessing.LabelEncoder()
y_train=train_labels
y_train = tf.keras.utils.to_categorical(lb.fit_transform(y_train))

In [ ]:
SAMPLE_RATE = 44100
def get_mfcc(name):
    b, _ = librosa.core.load(name, sr = SAMPLE_RATE)
    assert _ == SAMPLE_RATE
    try:
        mfcc = librosa.feature.mfcc(b, sr = SAMPLE_RATE, n_mfcc=40)
        #mfcc_delta = librosa.feature.delta(mfcc, width=5, order=1)
        return np.array(np.mean(mfcc, axis=1))
    except:
        print('bad file')
        return np.array([0]*40)

In [ ]:
SAMPLE_RATE = 44100
def get_delta(name):
    b, _ = librosa.core.load(name, sr = SAMPLE_RATE)
    assert _ == SAMPLE_RATE
    try:
        mfcc = librosa.feature.mfcc(b, sr = SAMPLE_RATE, n_mfcc=40)
        mfcc_delta = librosa.feature.delta(mfcc, width=5, order=1)
        return np.array(np.mean(mfcc_delta, axis=1))
    except:
        print('bad file')
        return np.array([0]*40)

In [ ]:
temp = get_delta("/content/drive/MyDrive/ADReSS-IS2020-train/ADReSS-IS2020-data/train/Full_wave_enhanced_audio/cc/S001.wav")
temp.shape
                

(40,)

In [ ]:
x = []
y = []

for filename in os.listdir('/content/drive/MyDrive/ADReSS-IS2020-train/ADReSS-IS2020-data/train/Full_wave_enhanced_audio/cc'):
    df = get_mfcc("/content/drive/MyDrive/ADReSS-IS2020-train/ADReSS-IS2020-data/train/Full_wave_enhanced_audio/cc/" + filename)
    x.append(df)
    y.append(0)
    
for filename in os.listdir('/content/drive/MyDrive/ADReSS-IS2020-train/ADReSS-IS2020-data/train/Full_wave_enhanced_audio/cd'):
    df = get_mfcc("/content/drive/MyDrive/ADReSS-IS2020-train/ADReSS-IS2020-data/train/Full_wave_enhanced_audio/cd/" + filename)
    x.append(df)
    y.append(1)

In [ ]:
xd = []
yd = []

for filename in os.listdir('/content/drive/MyDrive/ADReSS-IS2020-train/ADReSS-IS2020-data/train/Full_wave_enhanced_audio/cc'):
    df = get_delta("/content/drive/MyDrive/ADReSS-IS2020-train/ADReSS-IS2020-data/train/Full_wave_enhanced_audio/cc/" + filename)
    xd.append(df)
    yd.append(0)
    
for filename in os.listdir('/content/drive/MyDrive/ADReSS-IS2020-train/ADReSS-IS2020-data/train/Full_wave_enhanced_audio/cd'):
    df = get_delta("/content/drive/MyDrive/ADReSS-IS2020-train/ADReSS-IS2020-data/train/Full_wave_enhanced_audio/cd/" + filename)
    xd.append(df)
    yd.append(1)

In [ ]:
deldata = pd.DataFrame(xd)
label = pd.DataFrame(yd)
deldata['label'] = label
deldata.head()
deldata.shape

(108, 41)

In [ ]:
deldata = shuffle(deldata)
delata = deldata.reset_index(drop=True)
delata.head()

,0,1,2,3,4,5,6,7,8,9,...,31,32,33,34,35,36,37,38,39,label
0,0.000632,-0.000400,-0.004146,-0.005923,-0.004558,-0.006200,-0.008632,-0.006335,-0.001749,0.001643,...,-0.000843,-0.002226,-0.001621,-0.001511,-0.002546,-0.001936,-0.000967,-0.001464,-0.001925,0
1,0.011000,0.006950,-0.003551,-0.003685,-0.003250,-0.003663,-0.001474,-0.002147,-0.003461,-0.000712,...,0.001974,0.000857,-0.000569,-0.000107,0.000724,0.000743,0.000527,0.000456,0.000689,0
2,0.001470,0.001395,0.000376,-0.000271,-0.000282,-0.000122,-0.000070,-0.000311,-0.000763,-0.000662,...,-0.000264,-0.000474,-0.000284,0.000026,0.000315,0.000570,0.000225,-0.000235,-0.000442,1
3,0.018437,0.012034,-0.008067,-0.008874,0.000276,0.002677,0.002729,0.003687,0.002439,0.001354,...,0.000284,-0.000176,0.001540,0.001193,0.001041,-0.000297,-0.000246,0.000555,-0.000747,1
4,0.000047,0.000066,0.000065,0.000062,0.000059,0.000056,0.000051,0.000046,0.000041,0.000035,...,0.000008,0.000014,0.000020,0.000026,0.000032,0.000038,0.000043,0.000048,0.000052,1


In [ ]:
DX = delata.iloc[:,:-1]
DX.head()
DY=Y

In [ ]:
rfc = RandomForestClassifier(n_estimators = 150)
XD_train, XD_val, yD_train, yD_val = train_test_split(DX, DY, test_size=0.2, random_state=10, shuffle = True)
rfc.fit(DX, DY)

RandomForestClassifier(n_estimators=150)

In [ ]:
acc_delta=rfc.score(TDX, TDY)
print('Accuracy using only delta features: ',acc_delta)

Accuracy using only delta features:  0.4583333333333333


In [ ]:
l = linear_model.LogisticRegression(C=0.0001)
l.fit(DX,DY)

LogisticRegression(C=0.0001)

In [ ]:
accdelta=l.score(TDX, TDY)
print('Accuracy using only delta features: ',accdelta)

Accuracy using only delta features:  0.5208333333333334


In [ ]:
data = pd.DataFrame(x)
label = pd.DataFrame(y)
data['label'] = label
data.head()
data.shape

(108, 41)

In [ ]:
from sklearn.utils import shuffle
data = shuffle(data)
data = data.reset_index(drop=True)
data.head()

,0,1,2,3,4,5,6,7,8,9,...,31,32,33,34,35,36,37,38,39,label
0,-647.470154,73.837791,20.598610,11.832037,5.994508,4.896005,3.587691,-0.970787,-4.193769,-4.257558,...,-1.161308,-1.084687,-0.965439,-2.012323,-1.868908,-1.595058,-1.471887,-1.887323,-1.767366,0
1,-299.500977,170.659302,38.962574,1.235188,8.952155,6.384075,-5.129279,-9.316207,-6.096016,-4.574850,...,-1.422376,-1.994360,-2.675920,-3.078722,-2.360181,-0.881648,-0.024202,-0.341197,-1.377586,1
2,-644.097473,23.652416,13.427549,10.828506,8.314803,4.308354,1.919591,1.853076,0.648600,0.387064,...,-2.031844,-2.147707,-1.793273,-1.696349,-1.270723,-0.789062,-0.478098,-0.296390,-0.160531,0
3,-752.406494,52.879448,27.359529,14.342052,9.042205,6.971618,4.089694,3.891873,1.711816,2.228172,...,-0.411470,-0.571743,-0.071638,-0.248480,-0.207247,-0.210997,0.136996,0.401446,0.359582,1
4,-426.231018,77.847389,7.452454,7.383195,-3.602942,-5.444239,-3.694212,-8.580276,-8.932719,-1.927036,...,-1.926689,-2.147499,-1.916187,-1.342119,-1.098734,-0.718994,0.094379,0.389756,1.127997,1


In [ ]:
X = data.iloc[:,:-1]
X.head()

,0,1,2,3,4,5,6,7,8,9,...,30,31,32,33,34,35,36,37,38,39
0,-647.470154,73.837791,20.598610,11.832037,5.994508,4.896005,3.587691,-0.970787,-4.193769,-4.257558,...,-1.417511,-1.161308,-1.084687,-0.965439,-2.012323,-1.868908,-1.595058,-1.471887,-1.887323,-1.767366
1,-299.500977,170.659302,38.962574,1.235188,8.952155,6.384075,-5.129279,-9.316207,-6.096016,-4.574850,...,-1.579020,-1.422376,-1.994360,-2.675920,-3.078722,-2.360181,-0.881648,-0.024202,-0.341197,-1.377586
2,-644.097473,23.652416,13.427549,10.828506,8.314803,4.308354,1.919591,1.853076,0.648600,0.387064,...,-2.103207,-2.031844,-2.147707,-1.793273,-1.696349,-1.270723,-0.789062,-0.478098,-0.296390,-0.160531
3,-752.406494,52.879448,27.359529,14.342052,9.042205,6.971618,4.089694,3.891873,1.711816,2.228172,...,-1.226796,-0.411470,-0.571743,-0.071638,-0.248480,-0.207247,-0.210997,0.136996,0.401446,0.359582
4,-426.231018,77.847389,7.452454,7.383195,-3.602942,-5.444239,-3.694212,-8.580276,-8.932719,-1.927036,...,-1.798399,-1.926689,-2.147499,-1.916187,-1.342119,-1.098734,-0.718994,0.094379,0.389756,1.127997


In [ ]:
Y = data.iloc[:,-1]
Y.head()

0    0
1    1
2    0
3    1
4    1
Name: label, dtype: int64

In [ ]:
tx = []
ty = []

for filename in os.listdir('/content/drive/MyDrive/ADReSS-IS2020-test/ADReSS-IS2020-data/test/Full_wave_enhanced_audio/cc'):
    df = get_mfcc("/content/drive/MyDrive/ADReSS-IS2020-test/ADReSS-IS2020-data/test/Full_wave_enhanced_audio/cc/" + filename)
    tx.append(df)
    ty.append(0)
    
for filename in os.listdir('/content/drive/MyDrive/ADReSS-IS2020-test/ADReSS-IS2020-data/test/Full_wave_enhanced_audio/cd'):
    df = get_mfcc("/content/drive/MyDrive/ADReSS-IS2020-test/ADReSS-IS2020-data/test/Full_wave_enhanced_audio/cd/" + filename)
    tx.append(df)
    ty.append(1)

In [ ]:
txd = []
tyd = []

for filename in os.listdir('/content/drive/MyDrive/ADReSS-IS2020-test/ADReSS-IS2020-data/test/Full_wave_enhanced_audio/cc'):
    df = get_delta("/content/drive/MyDrive/ADReSS-IS2020-test/ADReSS-IS2020-data/test/Full_wave_enhanced_audio/cc/" + filename)
    txd.append(df)
    tyd.append(0)
    
for filename in os.listdir('/content/drive/MyDrive/ADReSS-IS2020-test/ADReSS-IS2020-data/test/Full_wave_enhanced_audio/cd'):
    df = get_delta("/content/drive/MyDrive/ADReSS-IS2020-test/ADReSS-IS2020-data/test/Full_wave_enhanced_audio/cd/" + filename)
    txd.append(df)
    tyd.append(1)

In [ ]:
tddata = pd.DataFrame(txd)
tdlabel = pd.DataFrame(tyd)
tddata['label'] = tdlabel
tddata.head()
tddata.shape

(48, 41)

In [ ]:
tddata = shuffle(tddata)
tddata = tddata.reset_index(drop=True)
tddata.head()

,0,1,2,3,4,5,6,7,8,9,...,31,32,33,34,35,36,37,38,39,label
0,0.000470,-0.000114,-0.001264,-0.001232,-0.000377,0.000205,0.000389,0.000493,0.000546,0.000542,...,-0.000129,0.000093,0.000263,0.000290,0.000203,7.888842e-07,-0.000194,-0.000220,-0.000176,0
1,-0.004654,-0.002868,0.003077,0.001917,0.001793,0.006102,0.004228,0.001938,0.001721,0.002608,...,-0.000280,-0.001866,-0.000527,0.000858,-0.001240,-1.239337e-03,0.000700,0.000296,-0.000087,0
2,-0.002171,-0.002010,-0.000181,0.000591,0.000401,0.000022,-0.000458,-0.000977,-0.001159,-0.000919,...,0.000563,0.000167,-0.000245,-0.000295,-0.000517,-8.128654e-04,-0.000524,0.000119,0.000344,1
3,-0.000144,0.000711,0.000451,-0.000891,-0.001118,-0.000543,-0.000256,-0.000781,-0.000124,-0.000292,...,0.000153,0.000114,-0.000277,-0.000334,0.000150,1.026244e-04,-0.000130,0.000006,-0.000078,0
4,-0.006409,-0.005304,-0.000862,0.000269,0.000571,0.000413,0.000783,0.000307,-0.000037,0.000617,...,0.000228,-0.000365,-0.000188,0.000237,0.000004,1.331102e-04,0.000286,0.000164,-0.000247,0


In [ ]:
TDX = tddata.iloc[:,:-1]
TDY=TY
TDX.head()

,0,1,2,3,4,5,6,7,8,9,...,30,31,32,33,34,35,36,37,38,39
0,0.000470,-0.000114,-0.001264,-0.001232,-0.000377,0.000205,0.000389,0.000493,0.000546,0.000542,...,-0.000132,-0.000129,0.000093,0.000263,0.000290,0.000203,7.888842e-07,-0.000194,-0.000220,-0.000176
1,-0.004654,-0.002868,0.003077,0.001917,0.001793,0.006102,0.004228,0.001938,0.001721,0.002608,...,0.000030,-0.000280,-0.001866,-0.000527,0.000858,-0.001240,-1.239337e-03,0.000700,0.000296,-0.000087
2,-0.002171,-0.002010,-0.000181,0.000591,0.000401,0.000022,-0.000458,-0.000977,-0.001159,-0.000919,...,0.000250,0.000563,0.000167,-0.000245,-0.000295,-0.000517,-8.128654e-04,-0.000524,0.000119,0.000344
3,-0.000144,0.000711,0.000451,-0.000891,-0.001118,-0.000543,-0.000256,-0.000781,-0.000124,-0.000292,...,0.000019,0.000153,0.000114,-0.000277,-0.000334,0.000150,1.026244e-04,-0.000130,0.000006,-0.000078
4,-0.006409,-0.005304,-0.000862,0.000269,0.000571,0.000413,0.000783,0.000307,-0.000037,0.000617,...,0.000282,0.000228,-0.000365,-0.000188,0.000237,0.000004,1.331102e-04,0.000286,0.000164,-0.000247


In [ ]:

tdata = pd.DataFrame(tx)
tlabel = pd.DataFrame(ty)
tdata['label'] = tlabel
tdata.head()
tdata.shape

(48, 41)

In [ ]:
from sklearn.utils import shuffle
tdata = shuffle(tdata)
tdata = tdata.reset_index(drop=True)
tdata.head()

,0,1,2,3,4,5,6,7,8,9,...,31,32,33,34,35,36,37,38,39,label
0,-659.858826,15.518291,4.643536,2.474949,1.856934,1.326966,1.053260,0.016039,0.478516,1.610914,...,0.068605,-0.241742,-0.275036,0.168752,0.323615,0.296374,0.240232,0.366528,0.565608,1
1,-673.250427,74.049706,27.088451,19.070208,12.857368,4.738476,3.261811,2.099103,-0.832536,-1.572835,...,-0.988127,-0.267990,-1.338692,-1.927761,-1.018401,-0.963583,-1.333958,-1.866681,-1.776560,1
2,-526.153442,88.377609,7.719505,23.285154,7.209867,2.030739,0.640738,-10.820081,-11.471400,1.595716,...,-2.545892,-2.863963,-1.910514,-2.834326,-2.424511,-2.800462,-2.271753,-1.474033,-1.309104,0
3,-430.444550,115.154541,35.908588,22.195105,8.008975,2.676006,1.217171,-1.682811,-0.173634,-2.103630,...,-2.055184,-3.010881,-2.526923,-2.380335,-1.916101,-1.406517,-1.930398,-1.869083,-2.432397,0
4,-487.827911,105.509575,34.932629,21.757771,8.002418,-1.943404,-5.894741,-7.497775,-6.641140,-7.171933,...,-2.130994,-1.187432,-1.704373,-2.020617,-1.124170,-0.367220,-0.804883,-1.588557,-0.710653,0


In [ ]:
TX = tdata.iloc[:,:-1]
TX.head()

,0,1,2,3,4,5,6,7,8,9,...,30,31,32,33,34,35,36,37,38,39
0,-659.858826,15.518291,4.643536,2.474949,1.856934,1.326966,1.053260,0.016039,0.478516,1.610914,...,-0.143819,0.068605,-0.241742,-0.275036,0.168752,0.323615,0.296374,0.240232,0.366528,0.565608
1,-673.250427,74.049706,27.088451,19.070208,12.857368,4.738476,3.261811,2.099103,-0.832536,-1.572835,...,-1.839497,-0.988127,-0.267990,-1.338692,-1.927761,-1.018401,-0.963583,-1.333958,-1.866681,-1.776560
2,-526.153442,88.377609,7.719505,23.285154,7.209867,2.030739,0.640738,-10.820081,-11.471400,1.595716,...,-4.535014,-2.545892,-2.863963,-1.910514,-2.834326,-2.424511,-2.800462,-2.271753,-1.474033,-1.309104
3,-430.444550,115.154541,35.908588,22.195105,8.008975,2.676006,1.217171,-1.682811,-0.173634,-2.103630,...,-1.594464,-2.055184,-3.010881,-2.526923,-2.380335,-1.916101,-1.406517,-1.930398,-1.869083,-2.432397
4,-487.827911,105.509575,34.932629,21.757771,8.002418,-1.943404,-5.894741,-7.497775,-6.641140,-7.171933,...,-3.290756,-2.130994,-1.187432,-1.704373,-2.020617,-1.124170,-0.367220,-0.804883,-1.588557,-0.710653


In [ ]:
TY = tdata.iloc[:,-1]
TY.head()

0    1
1    1
2    0
3    0
4    0
Name: label, dtype: int64

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

rfc = RandomForestClassifier(n_estimators = 150)
X_train, X_val, y_train, y_val = train_test_split(X, Y, test_size=0.2, random_state=10, shuffle = True)
rfc.fit(X_train, y_train)

RandomForestClassifier(n_estimators=150)

In [ ]:

acc_mfcc=rfc.score(TX, TY)
print('Accuracy using only MFCC: ',acc_mfcc)

Accuracy using only MFCC:  0.5625


In [ ]:
l = linear_model.LogisticRegression(C=0.0001)
l.fit(X_train,y_train)

LogisticRegression(C=0.0001)

In [ ]:
acc=l.score(TX, TY)
print('Accuracy using only MFCC: ',acc)

Accuracy using only MFCC:  0.5833333333333334


In [ ]:
def get_tdata(file_names,file_labels):
    features, features_delta, features_deltadelta, labels = np.empty((0,n_mfcc)), np.empty((0,n_mfcc)),np.empty((0,n_mfcc)), np.empty(0)
    for fn,fl in zip(file_names,file_labels):
        tpath='/content/drive/MyDrive/ADReSS-IS2020-train/ADReSS-IS2020-data/train/Full_wave_enhanced_audio/all'
        mfcc, mfcc_delta, mfcc_deltadelta = mfcc_extraction(fn,tpath)
        
        features = np.vstack([features, mfcc])
        features_delta = np.vstack([features_delta, mfcc_delta])
        features_deltadelta = np.vstack([features_deltadelta, mfcc_deltadelta])
        labels = np.append(labels, fl*np.ones(mfcc.shape[0]))
    return np.array(features), np.array(features_delta),np.array(features_deltadelta), np.array(labels, dtype = np.int)

In [ ]:

def get_testdata(file_names,file_labels):
    features, features_delta,features_deltadelta, labels = np.empty((0,n_mfcc)), np.empty((0,n_mfcc)),np.empty((0,n_mfcc)), np.empty(0)
    for fn,fl in zip(file_names,file_labels):
        tpath='/content/drive/MyDrive/ADReSS-IS2020-test/ADReSS-IS2020-data/test/Full_wave_enhanced_audio/all'
        mfcc, mfcc_delta, mfcc_deltadelta = mfcc_extraction(fn,tpath)
        
        features = np.vstack([features, mfcc])
        features_delta = np.vstack([features_delta, mfcc_delta])
        features_deltadelta = np.vstack([features_deltadelta, mfcc_deltadelta])
        labels = np.append(labels, fl*np.ones(mfcc.shape[0]))
    return np.array(features), np.array(features_delta),np.array(features_deltadelta), np.array(labels, dtype = np.int)

In [ ]:
n_mfcc=40

x_train1, x_train2,xtrain3,ytrain=get_tdata(train_fnames,labels_train)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  # This is added back by InteractiveShellApp.init_path()


MFCC only

In [ ]:
from sklearn.preprocessing import MinMaxScaler
data = x_train1
scaler = MinMaxScaler(feature_range=(0, 1))
scaler.fit(data)


print(np.amax(x_train1))

normalized_features = scaler.transform(data)
print(np.amax(normalized_features))

260.486083984375
1.0


In [ ]:
from sklearn import tree
clf = tree.DecisionTreeClassifier()

clf.fit(normalized_features, ytrain)

DecisionTreeClassifier()

In [ ]:
n_mfcc=40

x_test1, x_test2,ytest=get_testdata(test_fnames,labels_test)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  # Remove the CWD from sys.path while we load stuff.


In [ ]:
from sklearn.preprocessing import MinMaxScaler
data = x_test1
scaler = MinMaxScaler(feature_range=(0, 1))
scaler.fit(data)


print(np.amax(x_test1))

normalized_tfeatures = scaler.transform(data)
print(np.amax(normalized_tfeatures))

268.9161071777344
1.0


In [ ]:
y_pred = clf.predict(x_test1)
print(accuracy_score(ytest, y_pred))

0.5123048797637463


In [ ]:
y_pred = clf.predict(normalized_tfeatures)
print(accuracy_score(ytest, y_pred))

0.5434772418319036


In [ ]:
y_pred

array([0, 1, 0, 0, ..., 1, 1, 1, 1])

In [ ]:
y_pred.shape


(42666,)

delta only

In [ ]:
from sklearn.preprocessing import MinMaxScaler
data = x_train2
scaler = MinMaxScaler(feature_range=(0, 1))
scaler.fit(data)


print(np.amax(x_train2))

normalized_features_d = scaler.transform(data)
print(np.amax(normalized_features_d))
clfdelta = tree.DecisionTreeClassifier()

clfdelta.fit(normalized_features_d, ytrain)

178.9772491455078
1.0


DecisionTreeClassifier()

In [ ]:
from sklearn.preprocessing import MinMaxScaler
data = x_test2
scaler = MinMaxScaler(feature_range=(0, 1))
scaler.fit(data)


print(np.amax(x_test2))

normalized_tfeatures_d = scaler.transform(data)
print(np.amax(normalized_tfeatures_d))

174.30821228027344
1.0


In [ ]:
y_pred = clf.predict(normalized_tfeatures_d)
print(accuracy_score(ytest, y_pred))

0.42699104673510524


delta delta only

In [ ]:
from sklearn.preprocessing import MinMaxScaler
data = xtrain3
scaler = MinMaxScaler(feature_range=(0, 1))
scaler.fit(data)


print(np.amax(xtrain3))

normalized_features = scaler.transform(data)
print(np.amax(normalized_features))

94.35246276855469
1.0000000000000002


In [ ]:
from sklearn import tree
clf = tree.DecisionTreeClassifier()

clf.fit(normalized_features, ytrain)
n_mfcc=40

x_test1, x_test2,x_test3,ytest=get_testdata(test_fnames,labels_test)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if sys.path[0] == '':


In [ ]:
from sklearn.preprocessing import MinMaxScaler
data = x_test3
scaler = MinMaxScaler(feature_range=(0, 1))
scaler.fit(data)


print(np.amax(x_test3))

normalized_tfeatures = scaler.transform(data)
print(np.amax(normalized_tfeatures))

92.41328430175781
1.0


In [ ]:
y_pred = clf.predict(normalized_tfeatures)
print(accuracy_score(ytest, y_pred))

0.4881404396943702


In [ ]:
x_train1.shape

(83886, 40)

In [ ]:
x_train2.shape

(83886, 40)

In [ ]:
ytrain.shape

(83886,)

In [ ]:
ytrain[5]

0

In [ ]:
# Train classifier - MLP
print ("Training classifier...")
np.random.seed(42)
t0 = time()
clf1 = MLPClassifier(hidden_layer_sizes=(40), alpha=0.01)
clf1.fit(x_train1, ytrain)
#print ("Training on MFCC done in %0.3fs." % (time()-t0)
t0 = time()
clf2 = MLPClassifier(hidden_layer_sizes=(40), alpha=1)
clf2.fit(x_train2, ytrain)
#print "Training on MFCC delta done in %0.3fs." % (time()-t0)

Training classifier...


MLPClassifier(alpha=1, hidden_layer_sizes=40)

In [ ]:
x1_mfcc, x2_mfcc_delta = mfcc_extraction(test_fnames[45],testp)
     

In [ ]:
x1_mfcc.shape

(1041, 40)

In [ ]:
p=clf1.predict(x1_mfcc)

In [ ]:
print(p)

[1 0 0 1 ... 0 0 0 1]


In [ ]:
print(np.sum(p, axis=0))


835


In [ ]:
print(np.sum(p, axis=0).reshape(-1))

[797]


In [ ]:
testp='/content/drive/MyDrive/ADReSS-IS2020-test/ADReSS-IS2020-data/test/Full_wave_enhanced_audio/all'
def predict_proba(clf1, clf2, X_val):
    pred_proba = np.empty((0,2))
    for x in X_val:
        x_mfcc, x_mfcc_delta = mfcc_extraction(x,testp)
        y_pred_proba1 = np.sum(clf1.predict(x_mfcc), axis=0).reshape(-1)
        y_pred_proba2 = np.sum(clf2.predict(x_mfcc_delta), axis=0).reshape(-1)
        pred_proba = np.vstack([pred_proba,np.hstack([y_pred_proba1,y_pred_proba2])])
        #print pred_proba.shape
    return np.array(pred_proba, dtype=np.int)

In [ ]:
#pred_proba_val = predict_proba(clf1, clf2, files_val)
pred_proba_test = predict_proba(clf1, clf2, test_fnames)
print (pred_proba_test.shape)
#print (labels_val.shape)

(48, 2)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  # Remove the CWD from sys.path while we load stuff.


In [ ]:
pred_proba_test

array([[962, 960],
       [797, 219],
       [711, 653],
       [515, 380],
       ...,
       [424, 292],
       [835, 735],
       [602, 545],
       [447, 376]])

In [ ]:
#print "Score calibration..."
t0 = time()
logistic = linear_model.LogisticRegression(C=0.0001)
logistic.fit(pred_proba_test,labels_test)
#print "Done in %0.3fs." % (time()-t0)

LogisticRegression(C=0.0001)

In [ ]:
logistic = linear_model.LogisticRegression(C=0.0001)
y_test_pred = logistic.predict(pred_proba_test)
np.savetxt('y_test_pred_mfcc_delta_mfcc_mlp_logistic.txt', y_test_pred, fmt='%d')
print y_test_pred

In [ ]:
y_test_pred = logistic.predict(pred_proba_test)

In [ ]:
print(y_test_pred)

[1 0 1 0 0 0 0 0 0 1 0 1 1 0 0 0 0 0 0 0 0 1 0 0 1 1 0 1 0 1 0 1 1 0 1 1 1 1 1 0 1 0 0 1 0 1 1 0]


In [ ]:

from sklearn.metrics import accuracy_score
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import classification_report
print('Accuracy: {:.2f}'.format(accuracy_score(labels_test, y_test_pred)))

Accuracy: 0.69


In [ ]:
DATA = Path('/content/drive/MyDrive/ADReSS-IS2020-test')
TEST_AUDIO_PATH = DATA/'ADReSS-IS2020-data/test/Full_wave_enhanced_audio/cd'
TEST_AUDIO_PATH2 = DATA/'ADReSS-IS2020-data/test/Full_wave_enhanced_audio/cc'
TEST_IMAGE_PATH = DATA/'ADReSS2020_images/test'
test_cd = [f.name for f in (TEST_AUDIO_PATH.iterdir())
                         if 'S' in f.name]
len(test_cd)
test_cc = [f.name for f in (TEST_AUDIO_PATH2.iterdir())
                         if 'S' in f.name]


In [ ]:
len(test_cc)

24

In [ ]:
test_fnames=[]
for i in test_cc:
   test_fnames.append(i)
for i in test_cd:
   test_fnames.append(i)

In [ ]:
test_fnames


['S177.wav',
 'S161.wav',
 'S180.wav',
 'S201.wav',
 'S199.wav',
 'S166.wav',
 'S197.wav',
 'S193.wav',
 'S186.wav',
 'S175.wav',
 'S184.wav',
 'S178.wav',
 'S174.wav',
 'S163.wav',
 'S202.wav',
 'S170.wav',
 'S204.wav',
 'S160.wav',
 'S207.wav',
 'S196.wav',
 'S206.wav',
 'S183.wav',
 'S172.wav',
 'S195.wav',
 'S189.wav',
 'S185.wav',
 'S168.wav',
 'S179.wav',
 'S203.wav',
 'S173.wav',
 'S200.wav',
 'S190.wav',
 'S165.wav',
 'S194.wav',
 'S192.wav',
 'S187.wav',
 'S182.wav',
 'S181.wav',
 'S167.wav',
 'S164.wav',
 'S176.wav',
 'S162.wav',
 'S198.wav',
 'S191.wav',
 'S169.wav',
 'S188.wav',
 'S171.wav',
 'S205.wav']

In [ ]:
import glob
import numpy as np
tcontrol = glob.glob('/content/drive/MyDrive/ADReSS-IS2020-test/ADReSS-IS2020-data/test/Full_wave_enhanced_audio/cc/*.*')
tdementia = glob.glob('/content/drive/MyDrive/ADReSS-IS2020-test/ADReSS-IS2020-data/test/Full_wave_enhanced_audio/cd/*.*')

import tensorflow as tf
#data = []
labelst= []
for i in tcontrol:   
    labelst.append('Control')
for i in tdementia:   
    labelst.append('Dementia')

In [ ]:
labels_test = pd.factorize(labelst)[0]

In [ ]:
labels_test.shape

(48,)

In [ ]:
testp='/content/drive/MyDrive/ADReSS-IS2020-test/ADReSS-IS2020-data/test/Full_wave_enhanced_audio/all'
def predict_proba_mfcc(clf1, X_val):
    pred_proba = np.empty((0,1))
    for x in X_val:
        x_mfcc, x_mfcc_delta = mfcc_extraction(x,testp)
        y_pred_proba1 = np.sum(clf1.predict(x_mfcc), axis=0).reshape(-1)
        #y_pred_proba2 = np.sum(clf2.predict(x_mfcc_delta), axis=0).reshape(-1)
        pred_proba = np.vstack([pred_proba,y_pred_proba1])
        #print pred_proba.shape
    return np.array(pred_proba, dtype=np.int)
pred_proba_test_mfcc = predict_proba_mfcc(clf1,test_fnames)
print (pred_proba_test_mfcc.shape)

(48, 1)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  # Remove the CWD from sys.path while we load stuff.


In [ ]:

logist = linear_model.LogisticRegression(C=0.0001)
logist.fit(pred_proba_test_mfcc,labels_test)

LogisticRegression(C=0.0001)

In [ ]:
y_test_pred_mfcc = logistic.predict(pred_proba_test_mfcc)

ValueError: ignored

In [ ]:


from sklearn.metrics import accuracy_score
